# **ESC 20-SUMMER HW2**
### **19기 최우현**

---


### **1. 주석을 기반으로 하여 코딩을 해주세요.**

In [62]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pylab as plt
import random

In [63]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [64]:
#Setting Parameters
learning_rate = 0.1
training_epochs = 15
batch_size = 100

In [65]:
#Train and test sets by loading MNIST datasets 
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [66]:
#dataset loader에 train과 test할당하기(batch size, shuffle, drop_last 잘 설정할 것!)
train_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

test_loader = torch.utils.data.DataLoader(dataset=mnist_test,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          drop_last=True)

In [67]:
# Layer 쌓기 (조건: 3개의 Layer 사용, DropOut 사용 (p=0.3), ReLU 함수 사용, Batch normalization하기) 
# 각 Layer의 Hidden node 수 : 1st Layer (784,100), 2nd Layer(100,100),3rd Layer(100,10)
linear1 = torch.nn.Linear(784, 100, bias=True)
linear2 = torch.nn.Linear(100, 100, bias=True)
linear3 = torch.nn.Linear(100, 10, bias=True)

relu = torch.nn.ReLU()
bn1 = torch.nn.BatchNorm1d(100)
bn2 = torch.nn.BatchNorm1d(100)

drop_prob = 0.3
dropout = torch.nn.Dropout(p=drop_prob)

In [68]:
# xavier initialization을 이용하여 각 layer의 weight 초기화
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)

Parameter containing:
tensor([[-6.2633e-02, -1.3666e-02,  6.7819e-03,  2.1235e-01, -2.2802e-01,
          3.3828e-02,  3.7804e-02,  1.8646e-02,  1.5740e-01, -7.2078e-02,
          1.0993e-01,  6.3241e-02, -1.9329e-01, -1.5207e-01, -1.5088e-01,
         -7.3699e-02, -1.7913e-01, -1.8692e-02, -9.9205e-03,  2.7981e-02,
          1.4345e-01, -4.4702e-02,  9.2977e-02,  2.2748e-01,  1.3338e-01,
          3.2675e-02,  7.2531e-02, -5.5378e-02,  2.2212e-01, -1.2881e-01,
         -1.0136e-01, -1.6306e-01,  1.9431e-02, -1.6841e-01,  2.1120e-01,
          1.4543e-01,  1.0364e-01,  1.5119e-01,  1.2235e-01, -4.4031e-02,
         -1.6112e-01, -8.0274e-02, -1.9023e-01, -1.9396e-01,  5.7393e-02,
         -8.3063e-02,  1.7044e-01,  8.0732e-02,  1.0061e-01,  9.8878e-02,
         -2.2450e-01, -1.6809e-01, -2.1813e-01,  1.9226e-01, -1.4771e-01,
          3.0419e-02,  1.5642e-01, -1.4071e-01,  1.9636e-01, -8.8909e-02,
          5.4119e-02,  6.8158e-03,  2.1694e-01,  1.4856e-01,  1.6720e-01,
          9.2527

In [69]:
# torch.nn.Sequential을 이용하여 model 정의하기(쌓는 순서: linear-Batch Normalization Layer - ReLU- DropOut)
model = torch.nn.Sequential(linear1, bn1, relu, dropout,
                            linear2, bn2, relu, dropout,
                            linear3).to(device)

In [70]:
# Loss Function 정의하기 (CrossEntropy를 사용할 것!)
criterion = torch.nn.CrossEntropyLoss().to(device)

In [71]:
# optimizer 정의하기 (Adam optimizer를 사용할 것!)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [72]:
#cost 계산을 위한 변수 설정 
train_total_batch = len(train_loader)


In [73]:
#Training epoch (cost 값 초기 설정(0으로)과 model의 train 설정 꼭 할 것) 
model.train()    # set the model to train mode (dropout=True)

for epoch in range(training_epochs): 
  avg_cost = 0

  #train dataset을 불러오고(X,Y 불러오기), back propagation과 optimizer를 사용하여 loss를 최적화하는 코드
  for X, Y in train_loader:
    # reshape input image into [batch_size by 784]
    # label is not one-hot encoded
    X = X.view(-1, 28 * 28).to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()
    avg_cost += cost / train_total_batch
  
  print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))
print('Learning finished')


Epoch: 0001 cost = 0.499044180
Epoch: 0002 cost = 0.366370738
Epoch: 0003 cost = 0.327072978
Epoch: 0004 cost = 0.299702227
Epoch: 0005 cost = 0.301899821
Epoch: 0006 cost = 0.287576944
Epoch: 0007 cost = 0.281244099
Epoch: 0008 cost = 0.265668184
Epoch: 0009 cost = 0.264421642
Epoch: 0010 cost = 0.266991317
Epoch: 0011 cost = 0.262892544
Epoch: 0012 cost = 0.250128269
Epoch: 0013 cost = 0.255528241
Epoch: 0014 cost = 0.240182489
Epoch: 0015 cost = 0.229410902
Learning finished


In [74]:
#test data로 모델의 정확도를 검증하는 코드 (model의 evaluation mode 설정 꼭 할 것) #X_test 불러올 때 view를 사용하여 차원 변환할 것/ Y_test를 불러올때 labels사용 #accuracy의 초기 값 설정(0으로) 꼭 할 것
with torch.no_grad():
  model.eval()      # set the model to evaluation mode (dropout=False) 
  
  X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
  Y_test = mnist_test.test_labels.to(device)

  prediction = model(X_test)
  correct_prediction = torch.argmax(prediction, 1) == Y_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy:', accuracy.item())

  ##Test set에서 random으로 data를 뽑아 Label과 Prediction을 비교하는 코드
  r = random.randint(0, len(mnist_test)-1)
  X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 *28).float()
  Y_single_data = mnist_test.test_labels[r:r + 1]
  
  print('Label: ', Y_single_data.item())
  single_prediction = model(X_single_data)
  print('Prediction: ', torch.argmax(single_prediction, 1).item()) 

Accuracy: 0.9053000211715698
Label:  6
Prediction:  6


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")




---


### **2. 위에서 만든 모델에서 있던 Layer 들의 Hidden node 수를 증가 또는 감소(ex: 200, 300, 50...) 시켰을 때, train set에서의 cost와 test set에서 Accuracy가 기존 결과와 비교하였을 때 어떻게 달라졌는지 비교해주시면 됩니다.**


###784 -> 200 -> 150

In [75]:
linear4 = torch.nn.Linear(784, 200, bias=True)
linear5 = torch.nn.Linear(200, 150, bias=True)
linear6 = torch.nn.Linear(150, 10, bias=True)

relu = torch.nn.ReLU()
bn3 = torch.nn.BatchNorm1d(200)
bn4 = torch.nn.BatchNorm1d(150)
dropout = torch.nn.Dropout(p=drop_prob)

In [76]:
torch.nn.init.xavier_uniform_(linear4.weight)
torch.nn.init.xavier_uniform_(linear5.weight)
torch.nn.init.xavier_uniform_(linear6.weight)

Parameter containing:
tensor([[-0.0286, -0.0360,  0.1372,  ...,  0.0726, -0.1670, -0.1051],
        [-0.1543, -0.0325, -0.1490,  ...,  0.1577, -0.0049, -0.1190],
        [-0.0793,  0.1111,  0.0599,  ...,  0.0313, -0.0136, -0.1136],
        ...,
        [-0.1021,  0.1235, -0.1027,  ..., -0.0251,  0.1467, -0.1192],
        [ 0.0155,  0.1034,  0.1569,  ...,  0.1199,  0.1338,  0.1008],
        [ 0.1037, -0.0836, -0.0296,  ...,  0.1119,  0.0034,  0.0423]],
       requires_grad=True)

In [77]:
model1 = torch.nn.Sequential(linear4, bn3, relu, dropout, 
                             linear5, bn4, relu, dropout,
                             linear6).to(device)

In [78]:
# Loss Function 정의하기 (CrossEntropy를 사용할 것!)
criterion = torch.nn.CrossEntropyLoss().to(device)
# optimizer 정의하기 (Adam optimizer를 사용할 것!)
optimizer = torch.optim.Adam(model1.parameters(), lr=learning_rate) #model 설정 중요
#cost 계산을 위한 변수 설정 
train_total_batch = len(train_loader)

In [79]:
model1.train()    # set the model to train mode (dropout=True)

for epoch in range(training_epochs): 
  avg_cost = 0

  #train dataset을 불러오고(X,Y 불러오기), back propagation과 optimizer를 사용하여 loss를 최적화하는 코드
  for X, Y in train_loader:
    # reshape input image into [batch_size by 784]
    # label is not one-hot encoded
    X = X.view(-1, 28 * 28).to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = model1(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()
    avg_cost += cost / train_total_batch
  
  print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))
print('Learning finished')


Epoch: 0001 cost = 0.454042643
Epoch: 0002 cost = 0.327877909
Epoch: 0003 cost = 0.289008766
Epoch: 0004 cost = 0.270124257
Epoch: 0005 cost = 0.246979505
Epoch: 0006 cost = 0.233637109
Epoch: 0007 cost = 0.233835101
Epoch: 0008 cost = 0.229399025
Epoch: 0009 cost = 0.215847611
Epoch: 0010 cost = 0.218724027
Epoch: 0011 cost = 0.205324590
Epoch: 0012 cost = 0.199643523
Epoch: 0013 cost = 0.197351933
Epoch: 0014 cost = 0.192683548
Epoch: 0015 cost = 0.182401121
Learning finished


###784 -> 80 -> 40

In [80]:
with torch.no_grad():
  model1.eval()      # set the model to evaluation mode (dropout=False) 
  
  X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
  Y_test = mnist_test.test_labels.to(device)

  prediction = model1(X_test)
  correct_prediction = torch.argmax(prediction, 1) == Y_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy:', accuracy.item())

  ##Test set에서 random으로 data를 뽑아 Label과 Prediction을 비교하는 코드
  r = random.randint(0, len(mnist_test)-1)
  X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 *28).float()
  Y_single_data = mnist_test.test_labels[r:r + 1]
  
  print('Label: ', Y_single_data.item())
  single_prediction = model1(X_single_data)
  print('Prediction: ', torch.argmax(single_prediction, 1).item()) 

Accuracy: 0.902400016784668
Label:  3
Prediction:  3


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [81]:
linear7 = torch.nn.Linear(784, 80, bias=True)
linear8 = torch.nn.Linear(80, 40, bias=True)
linear9 = torch.nn.Linear(40, 10, bias=True)

relu = torch.nn.ReLU()
bn5 = torch.nn.BatchNorm1d(80)
bn6 = torch.nn.BatchNorm1d(40)

In [82]:
torch.nn.init.xavier_uniform_(linear7.weight)
torch.nn.init.xavier_uniform_(linear8.weight)
torch.nn.init.xavier_uniform_(linear9.weight)

Parameter containing:
tensor([[ 0.0764, -0.1672, -0.1104,  0.2413, -0.0343,  0.1766, -0.1721, -0.0776,
          0.1171, -0.1241, -0.0130,  0.1513, -0.3365,  0.2533,  0.0763,  0.1154,
         -0.2137, -0.2114,  0.1076,  0.2315, -0.1514, -0.0470, -0.2208, -0.1064,
         -0.3272,  0.2942,  0.1103, -0.1241, -0.2523,  0.2489,  0.2149,  0.0100,
          0.0939,  0.1920, -0.1599,  0.3283,  0.1579,  0.2843, -0.3044, -0.3180],
        [ 0.0129,  0.1536,  0.1610, -0.3339, -0.2555,  0.2820,  0.0388, -0.1521,
          0.1598,  0.2437, -0.0340, -0.1931, -0.0019,  0.0156, -0.2784, -0.0579,
          0.2416, -0.2166, -0.1497, -0.1625,  0.1780, -0.3412,  0.1921,  0.2808,
         -0.0559, -0.0684, -0.1461,  0.0419, -0.1616, -0.0360,  0.0019, -0.0437,
          0.2252, -0.3449,  0.0753, -0.0916, -0.2953, -0.1927,  0.1384, -0.0113],
        [-0.0143,  0.2981, -0.1526, -0.0247,  0.1032, -0.2590,  0.2343, -0.1993,
         -0.1258, -0.3448,  0.0600,  0.0269, -0.1728,  0.0507,  0.2754,  0.2248,
    

In [83]:
model2 = torch.nn.Sequential(linear7, bn5, relu, dropout, 
                             linear8, bn6, relu, dropout,
                             linear9).to(device)

In [84]:
# Loss Function 정의하기 (CrossEntropy를 사용할 것!)
criterion = torch.nn.CrossEntropyLoss().to(device)
# optimizer 정의하기 (Adam optimizer를 사용할 것!)
optimizer = torch.optim.Adam(model2.parameters(), lr=learning_rate)
#cost 계산을 위한 변수 설정 
train_total_batch = len(train_loader)

In [85]:
model2.train()    # set the model to train mode (dropout=True)

for epoch in range(training_epochs): 
  avg_cost = 0

  #train dataset을 불러오고(X,Y 불러오기), back propagation과 optimizer를 사용하여 loss를 최적화하는 코드
  for X, Y in train_loader:
    # reshape input image into [batch_size by 784]
    # label is not one-hot encoded
    X = X.view(-1, 28 * 28).to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = model2(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()
    avg_cost += cost / train_total_batch
  
  print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))
print('Learning finished')


Epoch: 0001 cost = 0.529607475
Epoch: 0002 cost = 0.386183470
Epoch: 0003 cost = 0.344602883
Epoch: 0004 cost = 0.332688093
Epoch: 0005 cost = 0.323567152
Epoch: 0006 cost = 0.302856177
Epoch: 0007 cost = 0.286634803
Epoch: 0008 cost = 0.288036227
Epoch: 0009 cost = 0.280324161
Epoch: 0010 cost = 0.279137641
Epoch: 0011 cost = 0.272234321
Epoch: 0012 cost = 0.266672462
Epoch: 0013 cost = 0.261246413
Epoch: 0014 cost = 0.262234926
Epoch: 0015 cost = 0.259336144
Learning finished


In [86]:
with torch.no_grad():
  model2.eval()      # set the model to evaluation mode (dropout=False) 
  
  X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
  Y_test = mnist_test.test_labels.to(device)

  prediction = model2(X_test)
  correct_prediction = torch.argmax(prediction, 1) == Y_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy:', accuracy.item())

  ##Test set에서 random으로 data를 뽑아 Label과 Prediction을 비교하는 코드
  r = random.randint(0, len(mnist_test)-1)
  X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 *28).float()
  Y_single_data = mnist_test.test_labels[r:r + 1]
  
  print('Label: ', Y_single_data.item())
  single_prediction = model2(X_single_data)
  print('Prediction: ', torch.argmax(single_prediction, 1).item()) 

Accuracy: 0.9509999752044678
Label:  5
Prediction:  5


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
